In [1]:
from models import HIPT_LGP_FC

from HIPT.HIPT_4K.hipt_4k import HIPT_4K
from HIPT.HIPT_4K.hipt_model_utils import get_vit256, get_vit4k, eval_transforms
from HIPT.HIPT_4K.hipt_heatmap_utils import *

import torch.nn.functional as F

In [2]:
def get_model():
    pretrained_weights256 = '/home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth'
    pretrained_weights4k = '/home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit4k_xs_dino.pth'

    if torch.cuda.is_available():
        device256 = torch.device('cuda:0')
        device4k = torch.device('cuda:1')
    else:
        device256 = torch.device('cpu')
        device4k = torch.device('cpu')


    ### ViT_256 + ViT_4K loaded independently (used for Attention Heatmaps)
    model256 = get_vit256(pretrained_weights=pretrained_weights256, device=device256)
    model4k = get_vit4k(pretrained_weights=pretrained_weights4k, device=device4k)

    ### ViT_256 + ViT_4K loaded into HIPT_4K API
    model = HIPT_4K(pretrained_weights256, pretrained_weights4k, device256, device4k)
    model.eval()
    return model

In [3]:
hipt4k=get_model()

Take key teacher in provided checkpoint dict
Pretrained weights found at /home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
# of Patches: 196
Take key teacher in provided checkpoint dict
Pretrained weights found at /home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit4k_xs_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Take key teacher in provided checkpoint dict
Pretrained weights found at /home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth and loaded with msg: _Incompatibl

In [4]:
hipt4k.eval()

region = Image.open('../HIPT/HIPT_4K/image_demo/image_4k.png')
x = eval_transforms()(region).unsqueeze(dim=0)

In [5]:
out = hipt4k.forward_asset_dict(x)

/usr/local/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


In [6]:
torch.tensor(out['features_cls256']).shape

torch.Size([256, 384])

In [7]:
model=HIPT_LGP_FC(freeze_4k=True, pretrain_4k='vit4k_xs_dino', n_classes=4)

# of Patches: 196
Loading Pretrained Local VIT model...
Done!
Freezing Pretrained Local VIT model
Done


In [8]:
model(torch.tensor(out['features_cls256']).unsqueeze(0))

(tensor([[-0.0900, -0.5607,  0.0396, -0.5876]], grad_fn=<AddmmBackward0>),
 tensor([[0.2966, 0.1853, 0.3377, 0.1804]], grad_fn=<SoftmaxBackward0>),
 tensor([[2]]),
 None,
 None)

In [9]:
model(torch.load('/project/GutIntelligenceLab/ss4yd/gtex_data/hipt4k_256cls_reps/GTEX-1117F-1026.pt'))

(tensor([[ 0.1260, -0.2569,  0.2892, -0.3795]], grad_fn=<AddmmBackward0>),
 tensor([[0.2888, 0.1969, 0.3400, 0.1742]], grad_fn=<SoftmaxBackward0>),
 tensor([[2]]),
 None,
 None)

In [10]:
cls_256=torch.load('/project/GutIntelligenceLab/ss4yd/gtex_data/hipt4k_256cls_reps/GTEX-111CU-0326.pt')

In [11]:
cls_256.shape

torch.Size([56, 256, 384])

In [12]:
model.eval()
model(cls_256)

(tensor([[ 0.1131, -0.2204, -0.0984, -0.4995]], grad_fn=<AddmmBackward0>),
 tensor([[0.3260, 0.2335, 0.2638, 0.1767]], grad_fn=<SoftmaxBackward0>),
 tensor([[0]]),
 None,
 None)

In [13]:
cls_256.unfold(1, 16, 16).transpose(1,2).shape

torch.Size([56, 384, 16, 16])

In [14]:
h_4096=model.local_vit(cls_256.unfold(1, 16, 16).transpose(1,2))
print(h_4096.shape)
h_4096 = model.global_phi(h_4096)
print(h_4096.shape)
h_4096 = model.global_transformer(h_4096.unsqueeze(1)).squeeze(1)
print(h_4096.shape)
A_4096, h_4096 = model.global_attn_pool(h_4096) 
print(A_4096.shape,h_4096.shape)
A_4096 = torch.transpose(A_4096, 1, 0)
print(A_4096.shape,h_4096.shape)
A_4096 = F.softmax(A_4096, dim=1)
print(A_4096.shape,h_4096.shape)
h_path = torch.mm(A_4096, h_4096)
print(h_path.shape)
h_WSI = model.global_rho(h_path)
print(h_WSI.shape)

logits = model.classifier(h_WSI)
logits

torch.Size([56, 192])
torch.Size([56, 192])
torch.Size([56, 192])
torch.Size([56, 1]) torch.Size([56, 192])
torch.Size([1, 56]) torch.Size([56, 192])
torch.Size([1, 56]) torch.Size([56, 192])
torch.Size([1, 192])
torch.Size([1, 192])


tensor([[ 0.1131, -0.2204, -0.0984, -0.4995]], grad_fn=<AddmmBackward0>)